In [4]:
import pandas as pd
import sqlite3

# ---------------------------
# 1. EXTRACT
# ---------------------------
df = pd.read_csv("data/students_raw.csv")

print("Dados brutos:")
display(df.head())

# ---------------------------
# 2. TRANSFORM
# ---------------------------

# Limpando nomes
df["nome"] = df["nome"].str.title().str.strip()

# Preenchendo valores faltantes
df["media_parcial"] = df["media_parcial"].fillna(0)
df["faltas"] = df["faltas"].fillna(0)

# Calculando média final simulada
df["media_final"] = df["media_parcial"] * 0.7 + (100 - df["faltas"]) * 0.3 / 10

# Classificação
def classify(row):
    if row["media_final"] >= 7:
        return "Aprovado"
    elif row["media_final"] >= 5:
        return "Recuperação"
    else:
        return "Reprovado"

df["status"] = df.apply(classify, axis=1)

# Criando um insight simples
df["insight"] = df["faltas"].apply(
    lambda x: "Risco alto por faltas" if x > 20
    else "Frequência aceitável"
)

print("Dados transformados:")
display(df.head())

# ---------------------------
# 3. LOAD
# ---------------------------

# Salvando CSV final
df.to_csv("output/students_processed.csv", index=False)

# Salvando no SQLite
conn = sqlite3.connect("output/students.db")
df.to_sql("students", conn, if_exists="replace", index=False)
conn.close()

print("✓ Pipeline concluído!")


Dados brutos:


,nome,disciplina,media_parcial,faltas
0,ana souza,Matemática,6.5,10
1,bruno santos,Biologia,8.0,5
2,carla mendes,Química,4.5,22
3,davi oliveira,Física,7.2,15
4,elisa lima,Matemática,9.0,3


Dados transformados:


,nome,disciplina,media_parcial,faltas,media_final,status,insight
0,Ana Souza,Matemática,6.5,10,7.25,Aprovado,Frequência aceitável
1,Bruno Santos,Biologia,8.0,5,8.45,Aprovado,Frequência aceitável
2,Carla Mendes,Química,4.5,22,5.49,Recuperação,Risco alto por faltas
3,Davi Oliveira,Física,7.2,15,7.59,Aprovado,Frequência aceitável
4,Elisa Lima,Matemática,9.0,3,9.21,Aprovado,Frequência aceitável


✓ Pipeline concluído!
